In [24]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 0 process data
bc = datasets.load_breast_cancer()
X_data, y_target = bc.data, bc.target

n_samples, n_features = X_data.shape

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = .2, random_state = 12)

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

X = torch.from_numpy(X_train_scaled.astype(np.float32))

X_test = torch.from_numpy(X_test_scaled.astype(np.float32))

y = torch.from_numpy(y_train.astype(np.float32))
y = y.view(y.shape[0], 1)

y_test = torch.from_numpy(y_test.astype(np.float32))
y_test = y_test.view(y_test.shape[0], 1)


#model
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_feature):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_feature, 1)
        
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

learning_rate = 0.01
model = LogisticRegression(n_features)

# loss and optimizer
criterion = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# training loop
epoch = 1000

for _ in range(epoch):
    #Forward pass
    y_pred = model(X)
    
    #loss and backward pass
    loss = criterion(y_pred, y)
    loss.backward()
    
    #update weight
    optimizer.step()

    #empty the gradients
    optimizer.zero_grad()
    
    if (_+1) % 100 == 0:
        print(f'epoch: {_+1}, loss = {loss.item():.4f}')


with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 100, loss = 0.2473
epoch: 200, loss = 0.1835
epoch: 300, loss = 0.1538
epoch: 400, loss = 0.1361
epoch: 500, loss = 0.1242
epoch: 600, loss = 0.1155
epoch: 700, loss = 0.1088
epoch: 800, loss = 0.1034
epoch: 900, loss = 0.0990
epoch: 1000, loss = 0.0953
accuracy = 0.9649
